## Exploring Audio Features

In [1]:
import sys
import os
import numpy as np
import pandas as pd
sys.path.insert(0, '../../')
from library.notebook_api.data_loader import  ModelDataLoader,CombinedDataLoader
from library.source_data.feature_extractor import AudioFeatureExtractor
from library.source_data.parallel_processor import AudioParallelProcessor
import librosa


In [2]:
#instantate data_loader and the dataframes it makes available 
data_loader = CombinedDataLoader('small')
df = data_loader.df
df_files_available = data_loader.df_files_available
df_genres_available = data_loader.df_genres_available
df_filtered = data_loader.df_filtered

tracks in meta 107574
tracks with files available in project_data_path:  9000
tracks with top level genres available 50598
tracks with genres and files (df_filtered) 9000


/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/exploratory/../../library/notebook_api/data_loader.py:28: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  self.df_filtered = self.df_files_available[self.df.label.isnull() ==False ]


In [3]:
audio = AudioFeatureExtractor(df_filtered.head(5))
audio.add_audio_data_to_df()
audio.add_numerical_features_to_df()
audio.add_mfcc_to_df()

/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/exploratory/../../library/source_data/feature_extractor.py:118: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)[0]


In [6]:
audio.df['mfcc'].apply(lambda vector: vector.shape)

0     (13, 2582)
2     (13, 2585)
3     (13, 2582)
15    (13, 2582)
16    (13, 2582)
Name: mfcc, dtype: object